# Prediction of tram delays in Cracow

## Dataset

There are 308,152 rows/records in the data that have been divided into almost equal parts:

-tram.train.h5 (175 986 rows)

-tram.test.h5 (132 166 rows)

<strong>This is real data.</strong>

In [213]:
import pandas as pd
import numpy as np
np.random.seed(0)

In [214]:
df_train = pd.read_hdf('../input/tram.train.h5')
df_test = pd.read_hdf('../input/tram.test.h5') #dataset without labels

## Concat datasets

Concatting `df_train` with ` df_test` and signing it to `df`.

In [215]:
df = pd.concat([df_train, df_test])
df.shape

(308152, 11)

In [216]:
df.sample(8)

,id,delay,datetime,stop,stop_name,number,direction,planned_time,vehicle_id,trip_id,seq_num
46049,46049,180.0,2018-07-23 22:46:12,409,Centralna,22,Kombinat,2018-07-23 22:43:00,6.352185e+18,6351558574044899599,NaN
70448,70448,NaN,NaN,367,Francesco Nullo,14,Mistrzejowice,2018-07-24 14:09:00,6.352185e+18,6351558574044727817,18.0
82986,82986,NaN,NaN,89,Bronowice,4,Wzgórza K.,2018-07-24 18:33:00,6.352185e+18,6351558574044457485,4.0
260805,260805,0.0,2018-07-30 21:18:35,2691,Chmieleniec,11,Czerwone Maki P+R,2018-07-30 21:19:00,6.352185e+18,6351558574044655637,21.0
151230,151230,NaN,NaN,79,Plac Inwalidów,24,Kurdwanów P+R,2018-07-26 11:06:00,6.352185e+18,6351558574047583239,9.0
229003,229003,0.0,2018-07-30 08:50:34,320,Reymana,20,Cichy Kącik,2018-07-30 08:50:00,6.352185e+18,6351558574044835847,18.0
61238,61238,NaN,NaN,129,Cystersów,4,Wzgórza K.,2018-07-24 10:14:00,6.352185e+18,6351558574044482053,16.0
193317,193317,240.0,2018-07-27 11:01:29,567,Kuklińskiego,20,Mały Płaszów,2018-07-27 10:57:00,6.352185e+18,6351558574046678282,15.0


## Dealing with features (`feature engineering`)

Krakow is one of the largest cities in Poland, which can result in streetcar delays caused by rush hour - when there are longer waits for public transportation.There are different dynamics during rush hour.

A hypothesis arises to show the model directly what time it is. <strong> We pull this hour from `planned_time` - the variable tells us what time the planned tram ride was. </strong>

Using `pd.to_datetime` we map planned_time to a datetime type, which allows us to work with dates more easily.

In [217]:
df['planned_time'] = pd.to_datetime(df['planned_time'], format='%Y-%m-%d %H:%M:%S')
df['day'] = df['planned_time'].dt.dayofweek
df['month'] = df['planned_time'].dt.month
df['hour'] = df['planned_time'].dt.hour
df['minute'] = df['planned_time'].dt.minute
df['time_diff'] = df['planned_time'].diff()
df['time_diff'] = df['time_diff'].dt.total_seconds()
df['stop_diff'] = df['stop'].diff()

df['rush_hours'] = df['hour'].apply(lambda x: 1 if (x > 5 and x < 10) | (x > 14 and x < 19) else 0) # rush_hours
df['night_hours'] = df['hour'].apply(lambda x: 1 if (x < 6 or x > 21) else 0) # night_hours

df['vehicle_id_log'] = df['vehicle_id'].map(lambda x: np.log(x))
df['vehicle_id_cat'] = pd.factorize(df['vehicle_id_log'])[0]

df['stop_hour'] = df['stop']/df['hour']

## Missing Values

In [218]:
df.isna().sum()

id                     0
delay             132166
datetime          132166
stop                   0
stop_name              0
number                 0
direction              0
planned_time           0
vehicle_id             0
trip_id                0
seq_num             2382
day                    0
month                  0
hour                   0
minute                 0
time_diff              1
stop_diff              1
rush_hours             0
night_hours            0
vehicle_id_log         0
vehicle_id_cat         0
stop_hour              0
dtype: int64

<strong>The large number of 'NaN' values among 'delay ' and 'datetime' is due to the fact that we merged df_train with df_test, which was devoid of labels.</strong>

In [230]:
df['time_diff'] = df['time_diff'].fillna(0)
df['stop_diff'] = df['stop_diff'].fillna(0)

df['seq_num'] = df['seq_num'].fillna(0)

### Bus stops
There are probably some streetcar stops that are more frequently used and prone to delay.
For extracting more data, you can collect more information about delays occurring at specific stops (e.g., average value, standard deviation).
Let's also count how often no delay occurred at a particular stop (zero delay) - `count_zeros_stopname_delay`.

In [220]:
def df_group_delay(df_train, groupby_feats):
    agg_params = {
        "mean_{}_delay".format("_".join(groupby_feats)): ("delay", "mean"),
        "median_{}_delay".format("_".join(groupby_feats)): ("delay", "median"),
        "count_{}_delay".format("_".join(groupby_feats)): ("delay", "count"),
        "std_{}_delay".format("_".join(groupby_feats)): ("delay", "std"),
        "count_zeros_{}_delay".format("_".join(groupby_feats)): ("delay", lambda vals: len([x for x in vals if x == 0]) ),
        "prob_zeros_{}_delay".format("_".join(groupby_feats)): ("delay", lambda vals: np.mean([x == 0 for x in vals]) ),
    }
    
    return df_train[groupby_feats + ["delay"]].groupby(groupby_feats).agg(**agg_params).reset_index()

In [221]:
stop_name_delay = df_group_delay(df_train, ["stop_name"]) #Now let's use df_group_delay and that will be what we achieved recently.

if "mean_stop_name_delay" not in df: #concatting the main df with the dataframe in delays at specific tram stops.
    df = pd.merge(df, stop_name_delay, on="stop_name", how="left") 

Let's add the driving direction (meaning `direction`). <strong>The same stop, but you can go in different directions so it will be different cases.  </strong>

In [222]:
stop_name_direction_delay = df_group_delay(df_train, ["stop_name", "direction"])
if "mean_stopname_direction_delay" not in df:
    df = pd.merge(df, stop_name_direction_delay, on=["stop_name", "direction"], how="left")

## Select features

In addition, we can ignore some features, because if you check them more carefully, it turns out that they do not contribute much.

In [223]:
feats = df.select_dtypes("number").columns
black_list = ["id", "delay", "trip_id", 'trip_id_log', 'vehicle_id', 'super_seq', 'stop_name_cat', 'est_delay', 'minute', 'seq_num_^2_median', 
              'seq_num_tmp','no_delays_stops','seq_num_power2', 'trip_id_seq_num_number_m_frec','day_frec','trip_id_cat', 'stop_diff']
feats = [x for x in feats if x not in black_list]

feats

['stop',
 'number',
 'seq_num',
 'day',
 'month',
 'hour',
 'time_diff',
 'rush_hours',
 'night_hours',
 'vehicle_id_log',
 'vehicle_id_cat',
 'stop_hour',
 'mean_stop_name_delay',
 'median_stop_name_delay',
 'count_stop_name_delay',
 'std_stop_name_delay',
 'count_zeros_stop_name_delay',
 'prob_zeros_stop_name_delay',
 'mean_stop_name_direction_delay',
 'median_stop_name_direction_delay',
 'count_stop_name_direction_delay',
 'std_stop_name_direction_delay',
 'count_zeros_stop_name_direction_delay',
 'prob_zeros_stop_name_direction_delay']

In [224]:
df_train = df[ df["delay"].notnull() ].copy().fillna(-1)
df_test = df[ df["delay"].isnull() ].copy().fillna(-1)

X_train = df_train[feats].fillna(-1).values
y_train = df_train["delay"].values
X_test = df_test[feats].fillna(-1).values

## Building the model

<strong>Due to the large amount of data/records, most of the models available in scikit-learn will not be able to learn the model!</strong> For that, we can still use the capabilities of the XGBoost library.

In [225]:
from sklearn.model_selection import cross_val_score
import xgboost as xgb

In [226]:
model = xgb.XGBRegressor(max_depth=5, n_estimators=50, random_state=0)

scores = cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

In [227]:
print(-np.mean(scores), np.std(scores))

31.03009691423145 0.9472878255639744


## Predicting on unseen data

In [228]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred[ y_pred < 0 ] = 0
df_test["delay"] = y_pred

In [229]:
df_test[ ["id", "delay"] ].to_csv('../output/simple_xgboost.csv', index=False) 